In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

In [2]:
batch_size = 100

# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transforms.ToTensor(), download=False)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [4]:
use_cuda = torch.cuda.is_available()                   
device = torch.device("cuda" if use_cuda else "cpu")   # use CPU or GPU

In [3]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()
        
        # encoder part
        self.fc1 = nn.Linear(x_dim, h_dim1)
        self.fc2 = nn.Linear(h_dim1, h_dim2)
        self.fc31 = nn.Linear(h_dim2, z_dim)
        self.fc32 = nn.Linear(h_dim2, z_dim)
        # decoder part
        self.fc4 = nn.Linear(z_dim, h_dim2)
        self.fc5 = nn.Linear(h_dim2, h_dim1)
        self.fc6 = nn.Linear(h_dim1, x_dim)
        
    def encoder(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return self.fc31(h), self.fc32(h) # mu, log_var
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # return z sample
        
    def decoder(self, z):
        h = F.relu(self.fc4(z))
        h = F.relu(self.fc5(h))
        return F.sigmoid(self.fc6(h)) 
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 784))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

# build model
vae = VAE(x_dim=784, h_dim1= 512, h_dim2=256, z_dim=2)

In [5]:
vae

VAE(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc31): Linear(in_features=256, out_features=2, bias=True)
  (fc32): Linear(in_features=256, out_features=2, bias=True)
  (fc4): Linear(in_features=2, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=512, bias=True)
  (fc6): Linear(in_features=512, out_features=784, bias=True)
)

In [6]:
optimizer = optim.Adam(vae.parameters())
# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

In [7]:
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [8]:
def test():
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data
            recon, mu, log_var = vae(data)
            
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [20]:
for epoch in range(1, 101):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 138.144844
Train Epoch: 1 [10000/60000 (17%)]	Loss: 135.876992
Train Epoch: 1 [20000/60000 (33%)]	Loss: 123.626523
Train Epoch: 1 [30000/60000 (50%)]	Loss: 130.468828
Train Epoch: 1 [40000/60000 (67%)]	Loss: 136.944961
Train Epoch: 1 [50000/60000 (83%)]	Loss: 135.150381
====> Epoch: 1 Average loss: 135.2993
====> Test set loss: 137.4759
Train Epoch: 2 [0/60000 (0%)]	Loss: 134.442490
Train Epoch: 2 [10000/60000 (17%)]	Loss: 135.831201
Train Epoch: 2 [20000/60000 (33%)]	Loss: 137.396943
Train Epoch: 2 [30000/60000 (50%)]	Loss: 133.153535
Train Epoch: 2 [40000/60000 (67%)]	Loss: 132.980752
Train Epoch: 2 [50000/60000 (83%)]	Loss: 124.777920
====> Epoch: 2 Average loss: 135.3926
====> Test set loss: 138.0603
Train Epoch: 3 [0/60000 (0%)]	Loss: 127.024365
Train Epoch: 3 [10000/60000 (17%)]	Loss: 130.144043
Train Epoch: 3 [20000/60000 (33%)]	Loss: 141.644160
Train Epoch: 3 [30000/60000 (50%)]	Loss: 133.979561
Train Epoch: 3 [40000/60000 (67%)]	Loss: 137.73

====> Test set loss: 136.5340
Train Epoch: 44 [0/60000 (0%)]	Loss: 131.969990
Train Epoch: 44 [10000/60000 (17%)]	Loss: 139.113994
Train Epoch: 44 [20000/60000 (33%)]	Loss: 131.471709
Train Epoch: 44 [30000/60000 (50%)]	Loss: 143.200654
Train Epoch: 44 [40000/60000 (67%)]	Loss: 133.793896
Train Epoch: 44 [50000/60000 (83%)]	Loss: 129.250400
====> Epoch: 44 Average loss: 133.1083
====> Test set loss: 137.2464
Train Epoch: 45 [0/60000 (0%)]	Loss: 139.311533
Train Epoch: 45 [10000/60000 (17%)]	Loss: 124.697666
Train Epoch: 45 [20000/60000 (33%)]	Loss: 134.626045
Train Epoch: 45 [30000/60000 (50%)]	Loss: 122.445801
Train Epoch: 45 [40000/60000 (67%)]	Loss: 143.381289
Train Epoch: 45 [50000/60000 (83%)]	Loss: 131.865635
====> Epoch: 45 Average loss: 133.3436
====> Test set loss: 136.8972
Train Epoch: 46 [0/60000 (0%)]	Loss: 133.606221
Train Epoch: 46 [10000/60000 (17%)]	Loss: 137.758281
Train Epoch: 46 [20000/60000 (33%)]	Loss: 139.329404
Train Epoch: 46 [30000/60000 (50%)]	Loss: 126.391230

Train Epoch: 86 [50000/60000 (83%)]	Loss: 137.143359
====> Epoch: 86 Average loss: 132.1437
====> Test set loss: 136.4259
Train Epoch: 87 [0/60000 (0%)]	Loss: 136.339326
Train Epoch: 87 [10000/60000 (17%)]	Loss: 128.276670
Train Epoch: 87 [20000/60000 (33%)]	Loss: 136.324414
Train Epoch: 87 [30000/60000 (50%)]	Loss: 122.100156
Train Epoch: 87 [40000/60000 (67%)]	Loss: 130.713613
Train Epoch: 87 [50000/60000 (83%)]	Loss: 130.453896
====> Epoch: 87 Average loss: 131.8683
====> Test set loss: 136.9362
Train Epoch: 88 [0/60000 (0%)]	Loss: 142.814111
Train Epoch: 88 [10000/60000 (17%)]	Loss: 127.472207
Train Epoch: 88 [20000/60000 (33%)]	Loss: 132.550098
Train Epoch: 88 [30000/60000 (50%)]	Loss: 129.870391
Train Epoch: 88 [40000/60000 (67%)]	Loss: 130.153320
Train Epoch: 88 [50000/60000 (83%)]	Loss: 136.818193
====> Epoch: 88 Average loss: 131.6422
====> Test set loss: 135.9716
Train Epoch: 89 [0/60000 (0%)]	Loss: 130.669482
Train Epoch: 89 [10000/60000 (17%)]	Loss: 118.477588
Train Epoch: 

In [21]:
with torch.no_grad():
    z = torch.randn(64, 2)
    sample = vae.decoder(z)
    
    save_image(sample.view(64, 1, 28, 28), './samples/sample_100' + '.png')